<font color='red'> Penalty kick application based on 'Professionals Play Minimax' by Ignacio Palacios-Huerta (2003). </font>

In [1]:
from bokeh.io import show, output_notebook
from bokeh.models import Plot, TapTool, ColumnDataSource, LabelSet, StaticLayoutProvider, Circle, MultiLine
from bokeh.models.widgets import Div
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.plotting import figure
from bokeh.models.renderers import GraphRenderer, GlyphRenderer
from bokeh.layouts import layout, row, column, gridplot
from bokeh.models import CustomJS, Button, DataTable, TableColumn, Rect
from bokeh.models.glyphs import Text
from bokeh.events import ButtonClick

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
bl_code = """
// resolve goalie playing Left

bl.disabled = true; // disable player actions
bm.disabled = true;
br.disabled = true;
b_next.disabled= false; // enable next button

// get strikers choice from last action
const foot = striker.text;

if (foot == 'left') {
    // get number of times 'left' subgame has been played
    n = parseInt(nleft.text)
    
    // if never played before
    if (n == 0) {
        const rand = Math.random(); // draw [0,1] RV
        goalprob rand = Math.random(); // draw [0,1] RV
        
        // play Left w.p 7/11 so goalie is indifferent
        if (rand < 7/11) {
            // resolve Left-play
            txt.data['text'][3] = 'Striker kicks Left; you jump Left!'
            txt.change.emit()
            // roll winning probability
            if (goalprob <= .67) {
                // resolve hit
                txt.data['text'][4] = 'Goal! (Win probability = 0.67)'
                txt.change.emit()                
            }
            // resolve miss
        }
        // else resolve Right-play
            
            txt.data['text'][3] = 'Striker kicks Right; you jump Left!'
            txt.change.emit()
    
    }
    // update number of times left subgame was played
    
} else if (foot == 'right') {
    // get number of times 'left' subgame has been played
    n = parseInt(nright.text)
    // if never played before
    if (n == 0) {
        const rand = Math.random(); // draw [0,1] RV
        // play Left w.p 7/11 so goalie is indifferent
        if (rand < 23/63) {
            // resolve Left-play
        
        }
    
    }
}

txt.data['text'][2] = 'Press Next to play another round.'
txt.change.emit()


const rand = Math.random(); // draw [0,1] RV
if (rand < 0.5) {
    // play left
    striker.text = 'left'
    const count = parseInt(nleft.text) + 1;
    nleft.text = count.toString();
    txt.data['text'][2] = 'Opponent chose a left-footed striker! Your move.'
    txt.change.emit()
} else {
    // play right
    striker.text = 'right'
    const count = parseInt(nright.text) + 1;
    nright.text = count.toString();
    txt.data['text'][2] = 'Opponent chose a right-footed striker! Your move.'
    txt.change.emit()
}
"""

### Define Figure and Sprites

In [4]:
p = figure(tools="", 
           toolbar_location=None, 
           title='FIFA 2020 Penalty Simulator',
           plot_width=400, plot_height=290, 
           x_range=(0, 100), y_range=(0, 90))

# hide axes and gridlines
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = None

# background color
p.background_fill_color = "green"

# goal posts and lines
p.multi_line([[24, 76, 78, 22, 24], [3,  12, 88, 97, 3 ], [34, 34, 66, 66]],
             [[63, 63, 47, 47, 63], [15, 63, 63, 15, 15], [63, 82, 82, 63]],
             color=["lightgreen", "lightgreen", "whitesmoke"],
             alpha=[1, 1, 1], line_width=4)

# striker box
p.quadratic(33, 15, 67, 15, 50, 2, color='lightgreen', line_width=4)

# striker sprite 
p.add_glyph(Circle(x=50, y=16, fill_color='lightblue', line_width=2, size=14)) # head
p.add_glyph(Rect(x=50, y=11, width=4, height=6, angle=0, fill_color='lightblue', line_width=2)) #body

# goalie sprite 
goalie_head = Circle(x=50, y=69, fill_color='red', line_width=2, size=12)
goalie_body = Rect(x=50, y=65, width=3, height=4, angle=0, fill_color='red', line_width=2)
p.add_glyph(goalie_head)
p.add_glyph(goalie_body)

# ball 
ball = Circle(x=50, y=20, fill_color='whitesmoke', line_width=2, size=12)
p.add_glyph(ball);

### Define Labels

In [5]:
scr_text = ColumnDataSource({'x': [2, 70, 2, 14, 14],
                             'y': [86, 86, 5, 40, 32],
                             'text': ['Rounds played: 00',
                                      'Total score: 00',
                                      '',
                                      '',
                                      '']})

labels = Text(x="x", y="y", text='text', text_color="whitesmoke",
              x_offset=0, y_offset=+9,
              text_baseline='ideographic', text_align='left')

p.add_glyph(scr_text, labels);

### Define State Variables as Divs

In [6]:
nround = Div(text='0', visible=False) # total number of rounds completed
score = Div(text='0', visible=False) # current score
freq_left_foot = Div(text='0,0,0', visible=False) # frequency of left, middle, and right kicks (left-footed)
freq_right_foot = Div(text='0,0,0', visible=False) # frequency of left, middle, and right kicks (right-footed)
kicker_foot = Div(text='', visible=False) # current footedness of the kicker

### Define Buttons

In [7]:
# start and next buttons
b_start = Button(label="Begin", button_type="success", width_policy='min')
b_next = Button(label="Next round", button_type="success", width_policy='min', disabled=True)

# right and left footed buttons
b_fl = Button(label="Left-Footed", button_type="success", width_policy='min', disabled=True)
b_fr = Button(label="Right-Footed", button_type="success", width_policy='min', disabled=True)

# right, middle, and left kick buttons
bl = Button(label="Left", button_type="success", width_policy='min', disabled=True, visible=False)
bm = Button(label="Middle", button_type="success", width_policy='min', disabled=True, visible=False)
br = Button(label="Right", button_type="success", width_policy='min', disabled=True, visible=False)

### Define Callbacks

In [8]:
bb_code = """
b_start.label = 'Restart';
bl.disabled = false;
bm.disabled = false;
br.disabled = false;

// make the computer choose an action
// this first choice is random
const rand = Math.random(); // draw [0,1] RV
if (rand < 0.5) {
    // play left
    striker.text = 'left'
    const count = parseInt(nleft.text) + 1;
    nleft.text = count.toString();
    txt.data['text'][2] = 'Opponent chose a left-footed striker! Your move.'
    txt.change.emit()
} else {
    // play right
    striker.text = 'right'
    const count = parseInt(nright.text) + 1;
    nright.text = count.toString();
    txt.data['text'][2] = 'Opponent chose a right-footed striker! Your move.'
    txt.change.emit()
}
"""
b_start_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr= b_fr, bl = bl, bm = bm, br = br, 
                                   b_next = b_next, labels=labels, txt=scr_text,
                                   nround = nround, score = score, freq_left_foot = freq_left_foot, 
                                   freq_right_foot = freq_right_foot, kicker_foot = kicker_foot),
                         code=bb_code)
b_start.js_on_click(b_start_click)

play_round = """
function play_round(choice) {

    // Increase the number of rounds by one
    const count = parseInt(rounds.text) + 1;
    rounds.text = count.toString(); 
}
"""

bl_code = """
play_round('left')
"""+play_round
bl_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr= b_fr, bl = bl, bm = bm, br = br, 
                                   b_next = b_next, labels=labels, txt=scr_text,
                                   nround = nround, score = score, freq_left_foot = freq_left_foot, 
                                   freq_right_foot = freq_right_foot, kicker_foot = kicker_foot),
                         code=bl_code)
bl.js_on_click(bl_click)

br_code = """
play_round('right')
"""+play_round
br_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr= b_fr, bl = bl, bm = bm, br = br, 
                                   b_next = b_next, labels=labels, txt=scr_text,
                                   nround = nround, score = score, freq_left_foot = freq_left_foot, 
                                   freq_right_foot = freq_right_foot, kicker_foot = kicker_foot),
                         code=br_code)
br.js_on_click(br_click)

bm_code = """
play_round('left')
"""+play_round
bm_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr= b_fr, bl = bl, bm = bm, br = br, 
                                   b_next = b_next, labels=labels, txt=scr_text,
                                   nround = nround, score = score, freq_left_foot = freq_left_foot, 
                                   freq_right_foot = freq_right_foot, kicker_foot = kicker_foot),
                         code=bm_code)
bm.js_on_click(bl_click)

### Format Layout and Show

In [9]:
#buttons_top = row(b1, b2, b3)
buttons_bot = row(b_start, nround, b_fl, b_fr, bl, bm, br, score, freq_left_foot, freq_right_foot, kicker_foot)
grid = gridplot([[p], [buttons_bot]], plot_width=400, plot_height=320)

show(grid)